In [ ]:
!pip install stanza

In [ ]:
#!/usr/bin/env python3
"""
project_with_stanza.py

Read pre‑standard Irish (or Scottish Gaelic/Manx) from STDIN, call the
Intergaelic standardiser API, parse the *standardised* text with Stanza,
and project the annotations back onto the original tokens.

Usage
-----
  python project_with_stanza.py [ga|gd|gv] > out.conllu
"""
import sys
import json
import urllib.parse
import urllib.request
import stanza                   # pip install stanza
from itertools import groupby   # for a very cheap sentence splitter

STD_API = "https://cadhan.com/api/intergaelic/3.0"

##############################################################################
# 1. call the standardiser ####################################################
##############################################################################

def standardise(text: str, lang: str):
    """
    Return a list of (orig_tok, std_tok) pairs from the Intergaelic API.
    """
    params = urllib.parse.urlencode({"foinse": lang, "teacs": text}).encode()
    headers = {"Content-Type": "application/x-www-form-urlencoded",
               "Accept":        "application/json"}
    req = urllib.request.Request(STD_API, params, headers=headers)
    with urllib.request.urlopen(req) as resp:
        return json.loads(resp.read())          # → [[orig, std], …]

##############################################################################
# 2. (very rough!) sentence boundary detection ###############################
##############################################################################

def naive_sentences(pairs):
    """
    Split the pair list into sentences whenever a token ends in . ! or ? .
    Stanza will be run in *tokenise‑pretokenised* mode, so sentences are
    lists‑of‑lists of *standardised* tokens.
    """
    sentence, current = [], []
    for orig, std in pairs:
        current.append((orig, std))
        if std.endswith((".", "!", "?")):
            sentence.append(current)
            current = []
    if current:
        sentence.append(current)
    return sentence

##############################################################################
# 3. parse with Stanza (pretok = True) #######################################
##############################################################################

def parse_standardised(sentences):
    """
    Run Stanza’s Irish pipeline on the already‑tokenised sentences and return
    the resulting Document object.
    """
    # Download once; afterwards this is a no‑op
    stanza.download("ga", processors="tokenize,pos,lemma,depparse", verbose=False)

    nlp = stanza.Pipeline(
        lang="ga",
        processors="tokenize,pos,lemma,depparse",
        tokenize_pretokenized=True,
        verbose=False,
    )
    # Convert to *only* the standard tokens for the parser
    std_sents = [[std for _orig, std in sent] for sent in sentences]
    return nlp(std_sents)

##############################################################################
# 4. project the annotations back ############################################
##############################################################################

def project(doc, sentences):
    """
    Yield CONLL‑U lines containing the original token but the Stanza annotation
    (lemma, UPOS, feats, head, deprel, …) taken from the aligned standard token.
    Assumes one‑to‑one alignment, which Scannell (2022) reports holds for ≈97 %
    of tokens; multi‑word tokens and many‑to‑one cases need extra work.
    """
    conllu_lines = []
    sent_id = 1
    for sent_pairs, sent_ann in zip(sentences, doc.sentences):
        conllu_lines.append(f"# sent_id = {sent_id}")
        conllu_lines.append(f"# text = {' '.join(orig for orig, _ in sent_pairs)}")
        for i, (pair, word) in enumerate(zip(sent_pairs, sent_ann.words), start=1):
            orig_tok, _ = pair
            # CoNLL‑U columns: ID, FORM, LEMMA, UPOS, XPOS, FEATS,
            #                  HEAD, DEPREL, DEPS, MISC
            line = [
                str(i),
                orig_tok,
                word.lemma or "_",
                word.upos  or "_",
                word.xpos  or "_",
                word.feats or "_",
                str(word.head) if word.head else "_",
                word.deprel or "_",
                "_",
                "_",
            ]
            conllu_lines.append("\t".join(line))
        conllu_lines.append("")   # blank line between sentences
        sent_id += 1
    return "\n".join(conllu_lines)

##############################################################################
# 5. driver ##################################################################
##############################################################################

def main():
    if len(sys.argv) != 2 or sys.argv[1] not in {"ga", "gd", "gv"}:
        sys.exit("Usage: python project_with_stanza.py [ga|gd|gv]")

    raw_text   = sys.stdin.read()
    pairs      = standardise(raw_text, sys.argv[1])
    sentences  = naive_sentences(pairs)
    doc        = parse_standardised(sentences)
    conllu_out = project(doc, sentences)
    print(conllu_out)

if __name__ == "__main__":
    main()
